In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import pandas as pd
import numpy as np 
import pickle
from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList

In [ ]:
pheno_file = '/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/Pheno/Paul/ukbb_omop_rolled_up_depth_4_closest_ancestor.csv'

In [ ]:
pheno_df = pd.read_csv(pheno_file)
eid_list = np.unique( np.array(list(pheno_df['eid'])))
nb_phenos = np.max(pheno_df['concept_id'])
res = np.zeros((len(eid_list), nb_phenos), dtype=int)
grouped = pheno_df.groupby('eid')



In [ ]:
data_list  = [np.array(grouped.get_group(eid)['concept_id'].unique())
                            for eid in eid_list]

for k, diseases in enumerate(data_list):
    res[k,np.array(diseases)-1] = 1
nb_max_diseases_sentence = max([len(sentence) for sentence in data_list])
columns = np.arange(1, nb_phenos+1)
pheno_df = pd.DataFrame(data=res, columns=columns)
pheno_df.insert(0, 'eid', eid_list)


pheno_df.set_index('eid', inplace=True)


In [ ]:
def get_paths_pheno_dicts(method, rollup_depth=None):
    if method == 'Paul':
        path_pheno_id_dict = f"/gpfs/commons/groups/gursoy_lab/pmeddeb/phenotype_embedding/vocab_dict/code2id_ukbb_omop_rolled_up_depth_{rollup_depth}_closest_ancestor.pickle"
        path_pheno_name_dict = f"/gpfs/commons/groups/gursoy_lab/pmeddeb/phenotype_embedding/vocab_dict/code2name_ukbb_omop_rolled_up_depth_{rollup_depth}_closest_ancestor.pickle"
        path_pheno_cat_dict = f"/gpfs/commons/groups/gursoy_lab/pmeddeb/phenotype_embedding/vocab_dict/code2cat_ukbb_omop_rolled_up_depth_{rollup_depth}_closest_ancestor.pickle"
    elif method == 'Abby':
        path_pheno_id_dict = '/gpfs/commons/groups/gursoy_lab/anewbury/embeddings/data/cohortId2ind.pickle'
        path_pheno_name_dict = '/gpfs/commons/groups/gursoy_lab/anewbury/embeddings/data/cohortId2name.pickle'
        path_pheno_cat_dict = '/gpfs/commons/groups/gursoy_lab/anewbury/embeddings/data/cohortId2name.pickle' #no separation between name and cat for Abby's method
    return path_pheno_id_dict, path_pheno_name_dict, path_pheno_cat_dict

In [ ]:
path_pheno_id_dict, path_pheno_name_dict, path_pheno_cat_dict = get_paths_pheno_dicts('Paul', 4)

In [ ]:
with open(path_pheno_id_dict, "rb") as f:
    pheno_id_dict = pickle.load(f)
with open(path_pheno_name_dict, "rb") as f:
    name_dict = pickle.load(f)
with open(path_pheno_cat_dict, "rb") as f:
    cat_dict = pickle.load(f)


In [ ]:
len(pheno_id_dict)

In [ ]:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Abby' # Paul, Abby
rollup_depth = 4
Classes_nb = 2 #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = False
save_data = False
remove_none = True
compute_features = True
padding = False
### data format
batch_size = 20
data_share = 1/10


In [ ]:
corr = np.corrcoef(tab, rowvar=False)

In [ ]:
corr.shape

In [ ]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=True, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         data_share=data_share,
                         prop_train_test=prop_train_test,
                         remove_none=True,
                         rollup_depth=rollup_depth)
patient_list = dataT.get_patientlist()


In [ ]:
data_tree = patient_list.get_tree_data()

In [ ]:
data_pheno_tree, data_label_tree = np.array(data_tree[0]), np.array(data_tree[1]).astype(int)

In [ ]:
nb_patients, nb_phenos = data_pheno_tree.shape

In [ ]:
corr = np.corrcoef(data_pheno_tree, rowvar=False)

In [ ]:
mask = np.tri(corr.shape[0], k=0, dtype=bool)
corr[mask] = None

In [ ]:
len(np.unique(np.where(corr > 0.8)[0]))/ nb_phenos

In [ ]:
np.sum(data_pheno_tree, axis=0).shape

In [ ]:
threshold = 50
nb_occ = np.sum(data_pheno_tree, axis=0)

In [ ]:
np.where(np.sum(data_pheno_tree, axis=0) <=threshold)

In [ ]:
len(np.argwhere(nb_occ <=threshold)), len(np.argwhere(np.sum(data_pheno_tree, axis=0) >= nb_patients - threshold))

In [ ]:
len(np.argwhere((nb_occ < threshold) & (counts_labels==0)))

In [ ]:
np.sum(counts_labels[np.sum(data_pheno_tree, axis=0) <=threshold] ==0 )

In [ ]:
def get_labels(col):
    return len(np.unique(data_label_tree[col==1]))-1
counts_labels = np.apply_along_axis(get_labels, axis =0, arr=data_pheno_tree)

In [ ]:
np.sum(counts_labels==-1)

In [ ]:
l = np.array([0]*int(400000*0.02) + [1]*int(400000*0.98))

In [ ]:
np.var(l)

In [ ]:
x = np.sum(data_pheno_tree == 1, axis =0)

In [ ]:
np.sum(x > 40508 *0.98)